## Components

In [8]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_chroma import Chroma

load_dotenv()

True

In [9]:

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=64,
    timeout=None,
    max_retries=2,
)

llm.invoke("Bonjour")

AIMessage(content="Bonjour ! Comment puis-je vous aider aujourd'hui ?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 36, 'total_tokens': 48, 'completion_time': 0.01119315, 'prompt_time': 0.001795188, 'queue_time': 0.088909779, 'total_time': 0.012988338}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7b3cfae3af', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--f78bc549-777a-40dd-9849-a2df9f936dc3-0', usage_metadata={'input_tokens': 36, 'output_tokens': 12, 'total_tokens': 48})

In [10]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

repo_id = "google/embeddinggemma-300m"

embeddings = HuggingFaceEndpointEmbeddings(
    repo_id=repo_id,
)

In [11]:
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

## Loading & processing docs

In [12]:
import pdfplumber
from langchain_core.documents import Document
import pandas as pd
import json

In [13]:
# Extraction des abréviations
ABBREVIATIONS = {
    "A": "Assistant",
    "ALSH": "Arts Lettres et Sciences Humaines",
    "CIDST": "Centre d'Information et de Documentation Scientifique et Technique",
    "CNARP": "Centre National d'Application de Recherches Pharmaceutiques",
    "CNRE": "Centre National de Recherche sur l'Environnement",
    "CNRIT": "Centre National de Recherches Industrielle et Technologique",
    "CNRO": "Centre National de Recherches Océanographiques",
    "CNTEMAD": "Centre National du Télé Enseignement de Madagascar",
    "ECD": "Employé de Courte Durée",
    "EFA": "Employé des Fonctionnaires Assimilés",
    "ELD": "Employé de Longue Durée",
    "ESUP": "Enseignement Supérieur",
    "F": "Féminin",
    "FOFIFA": "Foibem-pirenena momba ny Fikarohana ampiharina amin'ny Fampandrosoana ny eny Ambanivohitra",
    "IES": "Institut d'Enseignement Supérieur",
    "IMVAVET": "Institut Malgache des Vaccins Vétérinaires",
    "ISP": "Instituts Supérieurs Privés",
    "IST": "Institut Supérieur de Technologie",
    "L1": "Licence Première année",
    "L2": "Licence deuxième année",
    "L3": "Licence troisième année",
    "LMD": "Licence, Master, Doctorat",
    "M": "Masculin",
    "M1": "Master première année",
    "M2": "Master deuxième année",
    "MC": "Maître de Conférence",
    "P": "Professeur",
    "PAT": "Personnel Administratif et Technique",
    "PBZT": "Parc Botanique et Zoologique de Tsimbazaza",
    "PE": "Personnel Enseignant",
    "PIP": "Programme d'Investissement Public",
    "PT": "Professeur Titulaire",
    "PUB": "Public",
    "S/T": "Sous total",
    "SE": "Sciences de l'Education",
    "SI": "Sciences de l'Ingénieur",
    "SSANTE": "Sciences de la Santé",
    "SSTE": "Sciences de la Société",
    "STECH": "Sciences et Technologies",
    "TA": "Taux d'Abandon",
    "TP": "Taux de Promotion",
    "TR": "Taux de Redoublement"
}


with open('../data/abbreviations.json', 'w', encoding = "utf-8") as f:
    json.dump(ABBREVIATIONS, f, ensure_ascii=False, indent = 2)

In [14]:
# Configuration du chemin du PDF
data = "../data/MESUPRES_en_chiffres_MAJ.pdf"

In [15]:
import re

class MESUPRESChunker:
    """
    Chunker spécialisé pour MESUPRES
    Crée 3 types de chunks:
    1. Titre + Tableau complet
    2. Titre + Graphe + Analyse
    3. Titres seuls (pour questions sur sujets)
    """
    
    def __init__(self, pdf_path, abbreviations):
        self.pdf_path = pdf_path
        self.abbreviations = abbreviations
        self.chunks = []
    
    def extract_all_chunks(self):
        """Pipeline complet d'extraction"""
        
        with pdfplumber.open(self.pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, start=1):
                text = page.extract_text()
                if not text:
                    continue
                
                # 1. Extraire chunks Tableau
                table_chunks = self.extract_table_chunks(page, page_num, text)
                self.chunks.extend(table_chunks)
                
                # 2. Extraire chunks Graphe
                graph_chunks = self.extract_graph_chunks(page, page_num, text)
                self.chunks.extend(graph_chunks)
                
                # 3. Extraire titres seuls
                title_chunks = self.extract_title_chunks(text, page_num)
                self.chunks.extend(title_chunks)
        
        return self.chunks
    
    def extract_table_chunks(self, page, page_num, text):
        """
        Chunks Priorité 1: Titre + Tableau complet
        Format: "Tableau XX: TITRE\n[données du tableau]"
        """
        
        chunks = []
        
        # Détecter tous les tableaux de la page
        tables = page.extract_tables()
        
        if not tables:
            return chunks
        
        # Trouver les titres de tableaux dans le texte
        tableau_pattern = r'(Tableau\s+(\d+)\s*:\s*([^\n]+))'
        tableau_matches = list(re.finditer(tableau_pattern, text, re.IGNORECASE))
        
        for table_idx, table in enumerate(tables):
            if not table or len(table) < 2:
                continue
            
            # Trouver le titre correspondant
            title = ""
            table_number = ""
            
            if table_idx < len(tableau_matches):
                match = tableau_matches[table_idx]
                title = match.group(1)  # "Tableau 01: EVOLUTION..."
                table_number = match.group(2)  # "01"
            
            # Convertir le tableau en texte structuré
            table_text = self.table_to_structured_text(table)
            
            # Extraire aussi le contexte (phrase d'analyse après le tableau)
            analysis = self.extract_post_table_analysis(text, title)
            
            # Assembler le chunk complet
            chunk_content = f"{title}\n\n{table_text}"
            if analysis:
                chunk_content += f"\n\nAnalyse: {analysis}"
            
            # Extraire métadonnées
            keywords = self.extract_keywords(chunk_content)
            years = self.extract_years(chunk_content)
            regions = self.extract_regions(chunk_content)
            
            chunks.append(Document(
                page_content=chunk_content,
                metadata={
                    'page': page_num,
                    'type': 'table',
                    'id': f"Tableau {table_number}" if table_number else f"Tableau_p{page_num}_{table_idx}",
                    'keywords': keywords,
                    'years': years,
                    'regions': regions,
                    'source': 'MESUPRES'
                }
            ))
        
        return chunks
    
    def table_to_structured_text(self, table):
        """
        Convertir tableau en texte structuré
        Garde la structure ligne par ligne pour extraction précise
        """
        
        if not table or len(table) < 2:
            return ""
        
        lines = []
        
        # Headers (première ligne)
        headers = [str(cell).strip() if cell else '' for cell in table[0]]
        headers = [h if h else f"Col{i}" for i, h in enumerate(headers)]
        
        # Ligne de headers
        lines.append(" | ".join(headers))
        lines.append("-" * 80)  # Séparateur
        
        # Lignes de données
        for row in table[1:]:
            row_clean = [str(cell).strip() if cell else '' for cell in row]
            
            # Ignorer lignes vides
            if not any(row_clean):
                continue
            
            # Format: "Col1 | Col2 | Col3"
            lines.append(" | ".join(row_clean))
        
        return "\n".join(lines)
    
    def extract_post_table_analysis(self, text, table_title):
        """
        Extraire la phrase d'analyse qui suit le tableau
        Ex: "Le nombre d'inscrits a enregistré une baisse de 12%..."
        """
        
        if not table_title:
            return ""
        
        # Chercher le texte après le titre du tableau
        try:
            title_pos = text.index(table_title)
            after_title = text[title_pos + len(table_title):title_pos + 500]
            
            # Prendre la première phrase complète
            sentences = re.split(r'[.!?]\s+', after_title)
            if sentences and len(sentences[0]) > 20:
                return sentences[0].strip()
        except:
            pass
        
        return ""
    
    def extract_graph_chunks(self, page, page_num, text):
        """
        Chunks Priorité 2: Titre Graphe + Analyse
        Les graphes ne sont pas des images, mais des descriptions textuelles
        """
        
        chunks = []
        
        # Pattern pour détecter les graphes
        graphe_pattern = r'(Graphe?\s+(\d+)\s*:\s*([^\n]+))'
        matches = re.finditer(graphe_pattern, text, re.IGNORECASE)
        
        for match in matches:
            title = match.group(1)  # "Graphe 13: Taux de répartition..."
            graph_number = match.group(2)  # "13"
            
            # Extraire le contexte autour (200 chars avant et après)
            start = max(0, match.start() - 200)
            end = min(len(text), match.end() + 500)
            context = text[start:end]
            
            # Métadonnées
            keywords = self.extract_keywords(context)
            years = self.extract_years(context)
            regions = self.extract_regions(context)
            
            chunks.append(Document(
                page_content=context,
                metadata={
                    'page': page_num,
                    'type': 'graph',
                    'id': f"Graphe {graph_number}",
                    'keywords': keywords,
                    'years': years,
                    'regions': regions,
                    'source': 'MESUPRES'
                }
            ))
        
        return chunks
    
    def extract_title_chunks(self, text, page_num):
        """
        Chunks Priorité 3: Titres seuls
        Pour répondre aux questions "Quel est le sujet de..."
        """
        
        chunks = []
        
        # Tableaux
        for match in re.finditer(r'(Tableau\s+\d+\s*:\s*[^\n]+)', text, re.IGNORECASE):
            title = match.group(1).strip()
            chunks.append(Document(
                page_content=title,
                metadata={
                    'page': page_num,
                    'type': 'title',
                    'element_type': 'tableau',
                    'source': 'MESUPRES'
                }
            ))
        
        # Graphes
        for match in re.finditer(r'(Graphe?\s+\d+\s*:\s*[^\n]+)', text, re.IGNORECASE):
            title = match.group(1).strip()
            chunks.append(Document(
                page_content=title,
                metadata={
                    'page': page_num,
                    'type': 'title',
                    'element_type': 'graphe',
                    'source': 'MESUPRES'
                }
            ))
        
        return chunks
    
    def extract_keywords(self, text):
        """Extraire mots-clés pertinents"""
        keywords = []
        
        # Niveaux d'études
        levels = ['L1', 'L2', 'L3', 'M1', 'M2', 'Doctorat', 'Licence', 'Master']
        keywords.extend([l for l in levels if l in text])
        
        # Type d'établissement
        types = ['PUBLIC', 'PRIVE', 'ENSEMBLE']
        keywords.extend([t for t in types if t in text])
        
        # Domaines
        for abbr in ['SI', 'ALSH', 'SSTE', 'STECH', 'SSANTE', 'SE']:
            if abbr in text:
                keywords.append(abbr)
        
        return list(set(keywords))
    
    def extract_years(self, text):
        """Extraire années"""
        years = re.findall(r'\b(20\d{2})\b', text)
        return sorted(list(set(map(int, years))))
    
    def extract_regions(self, text):
        """Extraire régions"""
        regions = [
            'Antananarivo', 'Antsiranana', 'Diego', 'Fianarantsoa',
            'Mahajanga', 'Toamasina', 'Toliara', 'Vatovavy Fitovinany',
            'Alaotra Mangoro', 'Analanjirofo', 'Anosy', 'Vakinankaratra'
        ]
        found = [r for r in regions if r in text]
        return found

In [16]:
# Charger les abréviations
with open('../data/abbreviations.json', 'r', encoding='utf-8') as f:
    ABBREVIATIONS = json.load(f)

In [17]:
# Créer le chunker et extraire les chunks
chunker = MESUPRESChunker(
    pdf_path=data,
    abbreviations=ABBREVIATIONS
)

print("🔄 Extraction des chunks...")
chunks = chunker.extract_all_chunks()
print(f"✅ {len(chunks)} chunks créés")

🔄 Extraction des chunks...
✅ 294 chunks créés
✅ 294 chunks créés


In [18]:
# Statistiques sur les chunks
types = [c.metadata['type'] for c in chunks]
print(f"\n📊 Répartition:")
print(f"   - Tableaux: {types.count('table')}")
print(f"   - Graphes: {types.count('graph')}")
print(f"   - Titres: {types.count('title')}")


📊 Répartition:
   - Tableaux: 35
   - Graphes: 99
   - Titres: 160
